In [7]:
import utils
import params
import random
import os
import numpy as np
import time

from scipy.sparse import vstack
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout
from keras.objectives import MSE, MAE
from keras.callbacks import EarlyStopping

In [2]:
EPOCHS = 1
INPUT_SHAPE = 369537
OUTPUT_SHAPE = 128

In [6]:
model = Sequential()
model.add(Dense(input_dim=INPUT_SHAPE, units=150,kernel_initializer='normal'))
model.add(Activation("relu"))
model.add(Dropout(0.50))
model.add(Dense(units=130,kernel_initializer='normal'))
model.add(Activation("relu"))
model.add(Dropout(0.50))
model.add(Dense(units=128,kernel_initializer='normal'))
model.compile(loss='mean_squared_error', optimizer='adam')

In [5]:
text_descriptors = vstack(np.load('TMP/text_descriptors.npy'))
img_descriptors = np.load('TMP/img_descriptors.npy')

In [16]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10)
train_log = model.fit(text_descriptors, img_descriptors, batch_size=256, nb_epoch=10, validation_split=0.1, callbacks=[early_stopping])

C:\Users\bldr1\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Train on 372701 samples, validate on 41412 samples
Epoch 1/10


KeyboardInterrupt: 

In [9]:
plt.plot(train_log.history["loss"], label="loss")
plt.plot(train_log.history["val_loss"], label="val_loss")
plt.legend()

NameError: name 'train_log' is not defined

In [13]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

In [14]:
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk
